In [1]:
from colossus.cosmology import cosmology
cosmo=cosmology.setCosmology('planck18')
import numpy as np
import Func
from cobaya.run import run
from getdist import loadMCSamples, plots

# 为了结果方便，令omega_m为Planck给出的值了

# data and fid

In [2]:
zz = np.array([0.15,0.45,0.75])
# error of (f*sigma8) at zz
err_rsd = np.array([0.037622781639503125,0.013120288894571081,0.00801293154199452])
err_rsd_ksz = np.array([0.0048570460593981195,0.0029804102835748773,0.0028959660223023967])

In [3]:
# Fiducial values  
w0_fid = -0.57779661
wa_fid = -1.31630327
omegam_fid = 0.34166733
sigma8_fid = 0.79547449
fs8 = Func.growthrate(Om0=omegam_fid,w0=w0_fid, wa=wa_fid,z=zz) * Func.sigma(sigma8=sigma8_fid, Om0=omegam_fid,w0=w0_fid, wa=wa_fid,z=zz)
ref=[omegam_fid,sigma8_fid,w0_fid,wa_fid]

# RSD

In [4]:
def da(z_array, w0, wa):
    results = []
    for z in z_array:
        int_z = np.linspace(0, z, 50)
        E = Func.Ez(int_z, 0.3416, w0, wa)
        integral = np.trapz(1./E, int_z)
        results.append(integral)
    return np.array(results)

def log_like1(sigma8,w0,wa):

    E = Func.Ez(zz,0.3416,w0,wa)
    E_fid = Func.Ez(zz,0.3416,w0_fid,wa_fid)

    D = da(zz, w0, wa)
    D_fid = da(zz, w0_fid, wa_fid)
    
    f_th = Func.growthrate(Om0=0.3416,w0=w0, wa=wa,z=zz)
    sigma8_th = Func.sigma(sigma8=sigma8, Om0=0.3416,w0=w0, wa=wa,z=zz)
    
    Chi_1 = (fs8 - (E*D)/(E_fid*D_fid)*sigma8_th*f_th)

    Chi_2 = err_rsd**2
    Chi_C = Chi_1**2/Chi_2
    Chi = np.sum(Chi_C)
    
    return -0.5*Chi

info = {"likelihood": {"loglike": log_like1}, \
        "params": {"w0": {"prior": {"min": -2., "max": 2.0},'ref': ref[2],"latex": r'w_0',"proposal": 0.001}, \
                    "wa": {"prior": {"min": -3., "max": 1.0},'ref': ref[3],"latex": r'w_a',"proposal": 0.001},\
                    "sigma8": {"prior": {"min": 0.5, "max": 1.0},'ref': ref[1],"latex": r'\sigma8',"proposal": 0.001}
                    }, \
        "sampler": {"mcmc": {"Rminus1_stop": 0.01, "max_tries": 10000},},\
        "output": "chains_rsd/rsd"
        }
# ,"proposal": 0.001
updated_info, sampler = run(info,force=True)

[output] Output to be read-from/written-into folder 'chains_rsd', with prefix 'rsd'
[output] Found existing info files with the requested output prefix: 'chains_rsd/rsd'
[output] Will delete previous products ('force' was requested).
[loglike] Initialized external likelihood.
[mcmc] Getting initial point... (this may take a few seconds)
[mcmc] Initial point: w0:-0.5777966, wa:-1.316303, sigma8:0.7954745
[model] Measuring speeds... (this may take a few seconds)
[model] Setting measured speeds (per sec): {loglike: 117.0}
[mcmc] Covariance matrix not present. We will start learning the covariance of the proposal earlier: R-1 = 30 (would be 2 if all params loaded).
[mcmc] Sampling!
[mcmc] Progress @ 2024-09-02 16:24:49 : 1 steps taken, and 0 accepted.
[mcmc] Learn + convergence test @ 120 samples accepted.
[mcmc]  - Acceptance rate: 0.923
[mcmc]  - Convergence of means: R-1 = 4.438329 after 96 accepted steps
[mcmc]  - Updated covariance matrix of proposal pdf.
[mcmc] Learn + convergence te

In [5]:
readsamps1 = loadMCSamples('chains_rsd/rsd')
cov_rsd = readsamps1.cov(['w0','wa','sigma8'])
print("cov is:")
print(cov_rsd)
fisher_rsd = np.linalg.inv(cov_rsd)
print("fisher is:")
np.array(fisher_rsd)

[root] *WARNING* outlier fraction 0.0003205128205128205 
cov is:
[[ 0.49229483 -0.239645    0.04809311]
 [-0.239645    1.28028046 -0.0061309 ]
 [ 0.04809311 -0.0061309   0.0056949 ]]
fisher is:


array([[ 1.71114915e+01,  2.52397287e+00, -1.41788328e+02],
       [ 2.52397287e+00,  1.15741656e+00, -2.00687738e+01],
       [-1.41788328e+02, -2.00687738e+01,  1.35138449e+03]])

# RSD and kSZ

In [7]:
def log_like2(sigma8,w0,wa):

    E = Func.Ez(zz,0.3416,w0,wa)
    E_fid = Func.Ez(zz,0.3416,w0_fid,wa_fid)

    D = da(zz, w0, wa)
    D_fid = da(zz, w0_fid, wa_fid)
    
    f_th = Func.growthrate(Om0=0.3416,w0=w0, wa=wa,z=zz)
    sigma8_th = Func.sigma(sigma8=sigma8, Om0=0.3416,w0=w0, wa=wa,z=zz)
    
    Chi_1 = (fs8 - (E*D)/(E_fid*D_fid)*sigma8_th*f_th)

    Chi_2 = err_rsd_ksz**2
    Chi_C = Chi_1**2/Chi_2
    Chi = np.sum(Chi_C)
    
    return -0.5*Chi

info = {"likelihood": {"loglike": log_like2}, \
        "params": {"w0": {"prior": {"min": -2., "max": 2.0},'ref': ref[2],"latex": r'w_0',"proposal": 0.001}, \
                    "wa": {"prior": {"min": -3., "max": 1.0},'ref': ref[3],"latex": r'w_a',"proposal": 0.001},\
                    "sigma8": {"prior": {"min": 0.5, "max": 1.0},'ref': ref[1],"latex": r'\sigma8',"proposal": 0.001}, 
                    }, \
        "sampler": {"mcmc": {"Rminus1_stop": 0.01, "max_tries": 10000},},\
        "output": "chains_rsd_ksz/rsd_ksz"
        }
# ,"proposal": 0.001
updated_info, sampler = run(info,force=True)

[output] Output to be read-from/written-into folder 'chains_rsd_ksz', with prefix 'rsd_ksz'
[output] Found existing info files with the requested output prefix: 'chains_rsd_ksz/rsd_ksz'
[output] Will delete previous products ('force' was requested).
[loglike] Initialized external likelihood.
[mcmc] Getting initial point... (this may take a few seconds)
[mcmc] Initial point: w0:-0.5777966, wa:-1.316303, sigma8:0.7954745
[model] Measuring speeds... (this may take a few seconds)
[model] Setting measured speeds (per sec): {loglike: 188.0}
[mcmc] Covariance matrix not present. We will start learning the covariance of the proposal earlier: R-1 = 30 (would be 2 if all params loaded).
[mcmc] Sampling!
[mcmc] Progress @ 2024-09-02 16:29:51 : 1 steps taken, and 0 accepted.
[mcmc] Learn + convergence test @ 120 samples accepted.
[mcmc]  - Acceptance rate: 0.857
[mcmc]  - Convergence of means: R-1 = 3.174295 after 96 accepted steps
[mcmc]  - Updated covariance matrix of proposal pdf.
[mcmc] Learn 

In [8]:
readsamps2 = loadMCSamples('chains_rsd_ksz/rsd_ksz')
cov_rsd_ksz = readsamps2.cov(['w0','wa','sigma8'])
print("cov is:")
print(cov_rsd_ksz)
fisher_rsd_ksz = np.linalg.inv(cov_rsd_ksz)
print("fisher is:")
np.array(fisher_rsd_ksz)

cov is:
[[ 1.25345376e-01 -2.05194292e-01  8.73882642e-03]
 [-2.05194292e-01  9.69058730e-01 -5.87065190e-03]
 [ 8.73882642e-03 -5.87065190e-03  7.50051440e-04]]
fisher is:


array([[ 3.06950329e+02,  4.54869568e+01, -3.22024254e+03],
       [ 4.54869568e+01,  7.82400579e+00, -4.68728658e+02],
       [-3.22024254e+03, -4.68728658e+02,  3.51834507e+04]])

# joint

In [8]:
g=plots.get_subplot_plotter(chain_dir=['/home/wangsy/mcmc/cpl/chains_rsd',
                                       '/home/wangsy/mcmc/cpl/chains_rsd_ksz',
                                       '/home/wangsy/chains_planck/base_w_wa/plikHM_TTTEEE_lowl_lowE_BAO'],
                                       width_inch=7)
roots = [ 'rsd','rsd_ksz','base_w_wa_plikHM_TTTEEE_lowl_lowE_BAO']
params = ['w0','wa','sigma8']
g.triangle_plot(roots, params, filled=True, legend_labels=['RSD','RSD+kSZ tomography', 'Planck'])
g.export('cpl1.pdf')


[root] *WARNING* 2D kernel density bandwidth optimizer failed for wa, omegam. Using fallback width: f(a) and f(b) must have different signs
